In [2]:
from langchain_core.tools import tool

from langchain_openai import ChatOpenAI
from prompts import sql_prompt
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain.agents.agent import AgentExecutor, RunnableMultiActionAgent
from langchain.agents import create_openai_tools_agent

gpt_3_5 = "gpt-3.5-turbo"
llm = ChatOpenAI(model=gpt_3_5, temperature=0)

ModuleNotFoundError: No module named 'prompts'

In [ ]:

db = SQLDatabase.from_uri("sqlite:///metadata.db")
prompt_template = ChatPromptTemplate.from_messages([
    ("system", sql_prompt),
    MessagesPlaceholder(variable_name="messages"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])
tools = tools = SQLDatabaseToolkit(llm=llm, db=db).get_tools()
agent = RunnableMultiActionAgent(
    runnable=create_openai_tools_agent(llm, tools, prompt_template),
    input_keys_arg=["messages"],
    return_keys_arg=["output"],
)
sql_agent_executor = AgentExecutor(
    name="SQL Agent Executor",
    agent=agent,
    tools=tools,
    callback_manager=None,
    verbose=True,
    max_iterations=15,
    max_execution_time=None,
    early_stopping_method="force",
)